This notebook will show you how to share an internet connection to the robot, if your laptop has more than one network interface (ie. wireless and wired cards)

#  Setup: Network's IP Addresses

The robot is connected directly to the router over `eth0`.

```bash
ubuntu@arm:~⟫ ifconfig | grep "addr"
eth0      Link encap:Ethernet  HWaddr 1c:ba:8c:a3:0f:22  
          inet addr:192.168.0.100  Bcast:255.255.255.255  Mask:255.255.255.0
          inet6 addr: fe80::1eba:8cff:fea3:f22/64 Scope:Link
          inet addr:127.0.0.1  Mask:255.0.0.0
          inet6 addr: ::1/128 Scope:Host
ubuntu@arm:~⟫ 


```

The Laptop is connected to a wired network providing internet access using `eth2`, and connected to the robot's router using `wlan2`.

```bash
ajc@felix420:~$ ifconfig | grep "addr"
eth2      Link encap:Ethernet  HWaddr 00:21:cc:bc:a0:6d  
          inet addr:10.31.0.28  Bcast:10.31.1.255  Mask:255.255.254.0
          inet6 addr: fe80::221:ccff:febc:a06d/64 Scope:Link
          inet addr:127.0.0.1  Mask:255.0.0.0
          inet6 addr: ::1/128 Scope:Host
vboxnet0  Link encap:Ethernet  HWaddr 0a:00:27:00:00:00  
          inet addr:192.168.56.1  Bcast:192.168.56.255  Mask:255.255.255.0
          inet6 addr: fe80::800:27ff:fe00:0/64 Scope:Link
wlan2     Link encap:Ethernet  HWaddr 10:0b:a9:b7:b8:90  
          inet addr:192.168.0.104  Bcast:192.168.0.255  Mask:255.255.255.0
          inet6 addr: fe80::120b:a9ff:feb7:b890/64 Scope:Link
ajc@felix420:~$ 

```

### In summary:
* Robot:
    * eth0: 192.168.0.100
* Laptop:
    * eth2: 10.31.0.28
    * wlan2: 192.168.0.104

# Setting the Robot's Gateway

On the robot:
```bash
ubuntu@arm:~⟫ sudo route add default gw 192.168.0.104
```

# Setting Laptop's Routing

On the laptop:

```bash
ajc@felix420:~$ sudo su
[sudo] password for ajc: 

root@felix420:/home/ajc# iptables --table nat --append POSTROUTING --out-interface eth2 -j MASQUERADE
root@felix420:/home/ajc# iptables --append FORWARD --in-interface wlan2 -j ACCEPT
root@felix420:/home/ajc# echo 1 > /proc/sys/net/ipv4/ip_forward
```

# Setting nameservers in `/etc/resolve.conf`

On the robot:
```bash
ubuntu@arm:~⟫ sudo nano /etc/resolv.conf
[sudo] password for ubuntu: 
```

Change the file to look like this:
```bash
# Dynamic resolv.conf(5) file for glibc resolver(3) generated by resolvconf(8)
#     DO NOT EDIT THIS FILE BY HAND -- YOUR CHANGES WILL BE OVERWRITTEN
#nameserver 192.168.0.254
domain localdomain
search localdomain
#nameserver 192.168.1.1
nameserver 192.168.0.104
nameserver 8.8.8.8
```

Congratulations, your robot should be connected to the internet!
```bash
ubuntu@arm:~⟫ ping google.com
PING google.com (216.58.192.46) 56(84) bytes of data.
64 bytes from nuq04s30-in-f14.1e100.net (216.58.192.46): icmp_seq=1 ttl=54 time=4.80 ms
64 bytes from nuq04s30-in-f14.1e100.net (216.58.192.46): icmp_seq=2 ttl=54 time=4.46 ms
64 bytes from nuq04s30-in-f14.1e100.net (216.58.192.46): icmp_seq=3 ttl=54 time=4.40 ms
64 bytes from nuq04s30-in-f14.1e100.net (216.58.192.46): icmp_seq=4 ttl=54 time=4.34 ms
64 bytes from nuq04s30-in-f14.1e100.net (216.58.192.46): icmp_seq=5 ttl=54 time=4.29 ms
64 bytes from nuq04s30-in-f14.1e100.net (216.58.192.46): icmp_seq=6 ttl=54 time=4.47 ms
64 bytes from nuq04s30-in-f14.1e100.net (216.58.192.46): icmp_seq=7 ttl=54 time=4.56 ms
^C
--- google.com ping statistics ---
7 packets transmitted, 7 received, 0% packet loss, time 6011ms
rtt min/avg/max/mdev = 4.291/4.477/4.803/0.168 ms
ubuntu@arm:~⟫ ^C
```

# References

[0] https://elementztechblog.wordpress.com/2014/12/22/sharing-internet-using-network-over-usb-in-beaglebone-black/